In [56]:
# prompt: create_folder_in_drive - define a function that create the folder /stock-rl in my google drive if it doesn't already exist. paramatrize the name of the folder

import os
from google.colab import drive
drive.mount('/content/drive')

def create_folder_in_drive(folder_name):
  """Creates a folder in Google Drive if it doesn't exist.

  Args:
    folder_name: The name of the folder to create.
  """
  drive_path = '/content/drive/My Drive/'  # Root directory in Google Drive
  folder_path = os.path.join(drive_path, folder_name)

  if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print(f"Folder '{folder_name}' created in Google Drive.")
  else:
    print(f"Folder '{folder_name}' already exists in Google Drive.")

# Example usage:
create_folder_in_drive('stock-rl')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Folder 'stock-rl' already exists in Google Drive.


In [67]:
# prompt: get_from_drive - define a function that get the file from google drive to local drive, under the folder /stock-rl. if the file already exists in the local drive, remove the old file. parametrize the file name and path name. do not change the name of the file

def get_from_drive(drive_filename, local_filename='', drive_folder='/stock-rl'):
  """Gets a file from Google Drive to the local drive, overwriting if it exists.

  Args:
    drive_filename: Name of the file in Google Drive.
    local_filename: Path to save the file locally.
    drive_folder: Folder in Google Drive to get from (default: '/stock-rl').
  """
  drive_path = '/content/drive/My Drive/' + drive_folder + '/' + drive_filename
  if not local_filename:
    local_filename = drive_filename

  if not os.path.exists(drive_path):
    print(f"File '{drive_filename}' does not exist in Google Drive.")
    return

  if os.path.exists(local_filename):
    os.remove(local_filename)
    print(f"Existing local file '{local_filename}' removed.")

  !cp "{drive_path}" "{local_filename}"
  print(f"File '{drive_filename}' retrieved from Google Drive to '{local_filename}'.")

# Example usage:
get_from_drive('stock_data.h5', 'stock_data.h5')
get_from_drive('stock_data_index.pkl', 'stock_data_index.pkl')


File 'stock_data.h5' retrieved from Google Drive to 'stock_data.h5'.
File 'stock_data_index.pkl' retrieved from Google Drive to 'stock_data_index.pkl'.


In [55]:
# prompt: load_dataframe_from_h5 - define a function that read the data from filename into a variable data2, and then restore the column names and row index from stock_data_index.pkl

import pandas as pd
def load_dataframe_from_h5(filename, index_filename):
  """Loads a pandas DataFrame from an HDF5 file and restores its index.

  Args:
    filename: The name of the HDF5 file containing the DataFrame data.
    index_filename: The name of the file containing the DataFrame index information.

  Returns:
    The loaded pandas DataFrame with restored index.
  """
  try:
    # Load the DataFrame data from the HDF5 file
    with h5py.File(filename, 'r') as hf:
      data2 = pd.DataFrame(hf['data'][:])

    # Load the index information from the pickle file
    with open(index_filename, 'rb') as f:
      index_info = pickle.load(f)

    # Restore the column names and row index
    data2.columns = index_info['columns']
    data2.index = index_info['index']

    print(f"DataFrame loaded from '{filename}' with restored index.")
    return data2

  except Exception as e:
    print(f"Error loading DataFrame: {e}")
    return None

# Example usage:
data2 = load_dataframe_from_h5('stock_data.h5', 'stock_data_index.pkl')
print(data2.head())

DataFrame loaded from 'stock_data.h5' with restored index.
Price      Adj Close                                                      \
Ticker        360.AX A2M.AX AAI.AX AFI.AX AGL.AX AIA.AX    ALD.AX ALL.AX   
Date                                                                       
1988-01-29       NaN    NaN    NaN    NaN    NaN    NaN  0.574315    NaN   
1988-02-01       NaN    NaN    NaN    NaN    NaN    NaN  0.574315    NaN   
1988-02-02       NaN    NaN    NaN    NaN    NaN    NaN  0.574315    NaN   
1988-02-03       NaN    NaN    NaN    NaN    NaN    NaN  0.574315    NaN   
1988-02-04       NaN    NaN    NaN    NaN    NaN    NaN  0.574315    NaN   

Price                           ... Volume                                     \
Ticker        ALQ.AX    AMC.AX  ... WEB.AX WES.AX WGX.AX WHC.AX WOR.AX WOW.AX   
Date                            ...                                             
1988-01-29  0.056859  0.313313  ...    NaN    0.0    NaN    NaN    NaN    NaN   
1988-02-

In [63]:
# prompt: save_dataframe_to_h5 - using h5py, define a function that store the dataframe on my hard drive. in the case where a filename already exists, remove the old file

import h5py

def save_dataframe_to_h5(dataframe, filename):
  """Saves a pandas DataFrame to an HDF5 file.

  Args:
    dataframe: The pandas DataFrame to save.
    filename: The name of the HDF5 file to create.
  """
  try:
    # Check if the file already exists and remove it
    if os.path.exists(filename):
      os.remove(filename)
      print(f"Existing file '{filename}' removed.")

    # Save the DataFrame to the HDF5 file
    with h5py.File(filename, 'w') as hf:
      hf.create_dataset('data', data=dataframe.to_numpy())
    print(f"DataFrame saved to '{filename}'.")

  except Exception as e:
    print(f"Error saving DataFrame: {e}")

# Example usage:
save_dataframe_to_h5(data, 'stock_data.h5')


DataFrame saved to 'stock_data.h5'.


In [64]:
# prompt: store_dataframe_index - using pickle, define a function that store the columns and row index of the dataframe

import pickle

def store_dataframe_index(dataframe, filename):
  """Stores the columns and row index of a pandas DataFrame using pickle.

  Args:
    dataframe: The pandas DataFrame.
    filename: The name of the file to store the index information.

  Remarks:
    The stored object contains these keys: 'columns', 'index'.
  """
  index_info = {
      'columns': dataframe.columns,
      'index': dataframe.index
  }
  with open(filename, 'wb') as f:
    pickle.dump(index_info, f)
  print(f"DataFrame index information saved to '{filename}'.")

# Example usage:
store_dataframe_index(data, 'stock_data_index.pkl')


DataFrame index information saved to 'stock_data_index.pkl'.


In [66]:
# prompt: save_to_drive - define a function that store stock_data.h5 and columns.pkl into my google drive, under the folder /stock-rl. if the file already exists in the google drive, remove the old file. parametrize the path name

def save_to_drive(local_filename, drive_filename='', drive_folder='/stock-rl'):
  """Saves a local file to Google Drive, overwriting if it exists.

  Args:
    local_filename: Path to the local file.
    drive_filename: Name of the file in Google Drive.
    drive_folder: Folder in Google Drive to save to (default: '/stock-rl').
  """
  drive_path = '/content/drive/My Drive/' + drive_folder + '/' + drive_filename
  if not drive_filename:
    drive_filename = local_filename

  if not os.path.exists(local_filename):
    print(f"Local file '{local_filename}' does not exist.")
    return

  if os.path.exists(drive_path):
    os.remove(drive_path)
    print(f"Existing file '{drive_filename}' removed from Google Drive.")

  !cp "{local_filename}" "{drive_path}"
  print(f"File '{local_filename}' saved to Google Drive as '{drive_filename}'.")

# Example usage:
save_to_drive('stock_data.h5', 'stock_data.h5')
save_to_drive('stock_data_index.pkl', 'stock_data_index.pkl')


Local file 'stock_data.h5' does not exist.
Local file 'stock_data_index.pkl' does not exist.
